# Process Carraro et al.

Note: This one has brutal batch effect (by lab); should just discard this one

## Set up Env

In [1]:
import os
import pandas as pd
import numpy as np
import scanpy as sc
from scipy.sparse import csr_matrix

In [2]:
from prep_utils import filter_samples, filter_celltypes, check_group_balance

In [3]:
dataset = 'carraro'
groupby = 'major'
sample_key = 'orig.ident'
condition_key = 'type'
batch_key = 'lab'

min_cells_per_sample = 700
sample_zcounts_max = 3
sample_zcounts_min = -2

# set filtering parameters
min_cells = 20 # min number of cells per cell type
min_samples = 5 # min number of samples that pass the threshold per cell type

## Preprocess

### Load data

In [ ]:
adata = sc.read_h5ad(os.path.join('data', f"{dataset}.h5ad"))
adata

In [ ]:
from scipy.sparse import issparse

In [ ]:
issparse(adata.X)

In [ ]:
adata.obs[[sample_key, condition_key]].drop_duplicates().nunique()

Filter samples, cell types, and genes

In [ ]:
adata = filter_samples(adata, sample_key, condition_key, min_cells_per_sample, sample_zcounts_max, sample_zcounts_min)

### Check Group balance

In [ ]:
adata = check_group_balance(adata, condition_key, sample_key)

In [ ]:
adata = filter_celltypes(adata=adata, groupby=groupby, sample_key=sample_key, min_cells=min_cells, min_samples=min_samples)

In [ ]:
# Remove genes expressed in few cells
sc.pp.filter_genes(adata, min_cells=min_cells)
adata

### Normalize

In [ ]:
adata.layers['counts'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

### Add Metadata

In [ ]:
adata.uns['sample_key'] = sample_key
adata.uns['batch_key'] = batch_key
adata.uns['condition_key'] = condition_key
adata.uns['groupby'] = groupby


In [ ]:
assert np.isin(['sample_key', 'batch_key', 'condition_key', 'groupby'], adata.uns_keys()).all()

## Run LIANA

In [ ]:
import liana as li

In [ ]:
li.mt.rank_aggregate.by_sample(adata, groupby=groupby, use_raw=False, sample_key=sample_key, verbose=True, n_perms=None)

### Write

In [ ]:
adata.write_h5ad(os.path.join('data', 'interim', f"{dataset}_processed.h5ad"))

## Run Dimensionality Reductions

In [ ]:
import os
import scanpy as sc
from classify_utils import dim_reduction_pipe

In [ ]:
adata = sc.read_h5ad(os.path.join('data', 'interim', f"{dataset}_processed.h5ad"), backed='r')

In [ ]:
dim_reduction_pipe(adata, dataset)

## Run Classifier

In [ ]:
import os
import scanpy as sc
from classify_utils import run_classifier

In [ ]:
adata = sc.read_h5ad(os.path.join('data', 'results', f"{dataset}_dimred.h5ad"), backed='r')

In [ ]:
run_classifier(adata, dataset)

In [ ]:
adata.uns['evaluate']

In [ ]:
adata.uns['auc'].sort_values(by=['score_key', 'reduction_name', 'fold'])